# Signing packages with Azure Trusted Signing

This was a bit more of a doozy than I would have liked. There's a lot of very manual setup, which isn't documented the best.



## Notebook Setup

Before running this notebook, make sure you're in the sln root:

In [ ]:
$gitRoot = git rev-parse --show-toplevel
cd $gitRoot

## Dependencies

We need both a PowerShell module for using `signtool`, and also the `az` CLI for auth

In [ ]:
Install-Module -Name TrustedSigning

Install azcli with winget, then restart the connection to refresh the path

In [ ]:
winget install --id Microsoft.AzureCLI

### Azure setup

In addition to those tools, you also need to do a bunch of setup on the Azure portal itself.

This link is actually a good place to start: [Quickstart: Set up Trusted Signing](https://learn.microsoft.com/en-us/azure/trusted-signing/quickstart?tabs=registerrp-portal%2Caccount-portal%2Corgvalidation%2Ccertificateprofile-portal%2Cdeleteresources-portal)

One thing that's critical, which I'm not sure is listed here: 
**You need to make sure to grant the following roles to someone**:
* Trusted Signing Identity Verifier
  * This is to be able to create a trusted signing identity
* Trusted Signing Certificate Profile Signer
  * This is to actually sign code with the cert profile you created

This is also probably the thing I did the most wrong. I'm sure you're not supposed to just grant that access to the one user you have, but the stakes are incredibly low for this test repo.

## Signing

Now that you've got all your deps installed, and you've got Azure an Trusted Certificate Account & Certificate Profile setup, you're ready to build some packages and sign them

### Build the packages

The simplest way I've found to build all the x64 packages is with the following dotnet command:

In [ ]:
$gitRoot = git rev-parse --show-toplevel
cd $gitRoot

dotnet build --configuration Release --no-restore -p Platform=x64 .\Extensions.sln

### Collect up the packages

Now, we're gonna stick all the packages into a single folder, for ease of signing.

In [ ]:
$gitRoot = git rev-parse --show-toplevel
cd $gitRoot

.\src\tools\Find-Msixs.ps1

### Logging in with az cli


Then, login with the following command. The first time I ran this, I had to manually select "Default directory" as the tenant. `az login` will print the tenant id, and loging with `az login --tenant TENANT_ID` instead

In [ ]:
az login

(I also need to manually select subscription 3, "Visual Studio Enterprise with MSDN")

## Actually doing the signing

Now that you're all signed in, and youve built the packages, and you've got them binplaced into `x64/tmp`, run the following:

In [ ]:
$gitRoot = git rev-parse --show-toplevel
cd $gitRoot

.\src\tools\Sign-Msixs.ps1